# Data Load 

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import warnings
# -------------------------------
# 0. Import required libraries
# -------------------------------
# For scaling and metrics
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error

# For ARIMA and SARIMAX
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX

# For Prophet
from prophet import Prophet

# For tree-based models
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# For visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from tqdm.notebook import tqdm

In [ ]:
df = pd.read_csv("c1_train.csv")

In [ ]:
time_order = ['Morning', 'Afternoon', 'Evening', 'Night']

# Convert the column
df['time_of_day'] = pd.Categorical(df['time_of_day'], categories=time_order, ordered=True)
df['plant_name'] = df['plant_name'].astype('category')
df['effectivedate'] = pd.to_datetime(df['effectivedate'])

In [ ]:
df.columns

Case 1: Provider 1 & Provider 2 + Supplement Forecast Data = Last FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 2: Provider 1 & Provider 2 + Supplement Forecast Data + Meteo Data = Last FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 3v1: Provider 1 & Provider 2 + Meteo Data = Dayahead FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 3v2: Provider 1 & Provider 2 = Dayahead FC
- Available Plants: A, B, C, D, E, F, G, H, J, K


In [ ]:
# Check if df exists before creating subsets
if 'df' in locals() and isinstance(df, pd.DataFrame) and not df.empty:
    df_c1 = df[['effectivedate', 'plant_name', 'production', 'capacity','year',
           'month', 'day', 'day_of_week', 'hour', 'hour_sin', 'hour_cos',
           'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
           'provider1_fc0', 'provider1_fc1200', 'provider1_fc40', 'provider1_fc55',
           'provider1_fc60', 'provider1_fc75', 'provider2_fc0', 'provider2_fc1200',
           'provider2_fc40', 'provider2_fc55', 'provider2_fc60', 'provider2_fc75',
           'provider1_ramp', 'provider1_speed_fc1200_to_fc75',
           'provider1_speed_fc75_to_fc60', 'provider1_speed_fc60_to_fc55',
           'provider1_speed_fc55_to_fc40', 'provider1_speed_fc40_to_fc0',
           'provider1_volatility', 'provider1_std_fc', 'provider1_mean_fc',
           'provider2_ramp', 'provider2_speed_fc1200_to_fc75',
           'provider2_speed_fc75_to_fc60', 'provider2_speed_fc60_to_fc55',
           'provider2_speed_fc55_to_fc40', 'provider2_speed_fc40_to_fc0',
           'provider2_volatility', 'provider2_std_fc', 'provider2_mean_fc',
           'season', 'is_weekend', 'time_of_day', 'week_of_year']].copy()

    df_c2 = df.copy() # Contains ALL original columns

    df_c3 = df[['effectivedate', 'plant_name', 'production', 'capacity',
           'temperature_2m', 'relative_humidity_2m', 'dew_point_2m',
           'apparent_temperature', 'precipitation', 'rain', 'snowfall',
           'snow_depth', 'weather_code_x', 'pressure_msl', 'surface_pressure',
           'cloud_cover', 'cloud_cover_low', 'cloud_cover_mid', 'cloud_cover_high',
           'et0_fao_evapotranspiration_x', 'vapour_pressure_deficit',
           'wind_speed_10m', 'wind_direction_10m', 'wind_gusts_10m',
           'et0_fao_evapotranspiration_y', 'temperature_2m_max',
           'temperature_2m_min', 'apparent_temperature_max',
           'apparent_temperature_min', 'daylight_duration', 'sunshine_duration',
           'precipitation_sum', 'rain_sum', 'snowfall_sum', 'precipitation_hours',
           'wind_speed_10m_max', 'wind_gusts_10m_max',
           'wind_direction_10m_dominant', 'shortwave_radiation_sum', 'year',
           'month', 'day', 'day_of_week', 'hour', 'hour_sin', 'hour_cos',
           'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
            'provider1_fc1200', 'provider2_fc1200', # Only fc1200 from providers
           'season', 'is_weekend', 'time_of_day', 'week_of_year',
           'weather_category_x', 'wind_u', 'wind_v', 'wind_gust_u', 'wind_gust_v',
           'wind_speed_squared', 'wind_speed_cubed', 'is_raining', 'is_snowing']].copy()

    df_c4 = df[['effectivedate', 'plant_name', 'production', 'capacity','year',
           'month', 'day', 'day_of_week', 'hour', 'hour_sin', 'hour_cos',
           'day_of_week_sin', 'day_of_week_cos', 'month_sin', 'month_cos',
           'provider1_fc1200', 'provider2_fc1200', # Only fc1200 from providers
           'season', 'is_weekend', 'time_of_day', 'week_of_year']].copy()
else:
    print("Warning: DataFrame 'df' not found or empty. Please load your data first.")
    # Create empty placeholders if needed for the script to run without error
    df_c1, df_c2, df_c3, df_c4 = pd.DataFrame(), pd.DataFrame(), pd.DataFrame(), pd.DataFrame()



In [ ]:
cases = {
    "case_1":df_c1,
    "case_2":df_c2,
    "case_3":df_c3,
    "case_4":df_c4,
}

In [ ]:
df.info()

# Training

This pipeline is suitable for testing "plant level" performances of the available models. It provides several customization options for the rolling window training methodology. Capacity factor forecasting can be adapted to this pipeline if "production to capacity factor" and "capacity factor to production" conversions handled externally. Capacity factor prediction with the whole data (including multiple plants) can be used as a POC for "regional power generation models" where power plants within the same regions who has similar production patterns can be forecasted together.

In [ ]:
# -------------------------------
# 1. Helper functions for data preparation
# -------------------------------
def prepare_features_for_regression(train_df, test_df, target_column, date_column):
    """
    Prepare training and test features:
      - Drop target and date columns.
      - One-hot encode categorical features.
      - Align train and test columns.
    """
    X_train = train_df.drop(columns=[target_column, date_column])
    X_test = test_df.drop(columns=[target_column, date_column])
    
    # One-hot encoding for categorical columns
    X_train = pd.get_dummies(X_train, drop_first=True)
    X_test = pd.get_dummies(X_test, drop_first=True)
    
    # Align columns between train and test (fill missing with 0)
    X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)
    
    y_train = train_df[target_column].values
    y_test = test_df[target_column].values
    
    return X_train, y_train, X_test, y_test

def prepare_regressor_data_for_prophet(df, regressor_columns):
    """
    For Prophet (with regressors), convert categorical features to numeric codes.
    """
    df = df.copy()
    for col in regressor_columns:
        if isinstance(df[col].dtype, pd.CategoricalDtype) or df[col].dtype == object:
            df[col] = df[col].astype('category').cat.codes
    return df

def get_categorical_columns(df, threshold=20):
    """
    Identify categorical columns in a DataFrame.
    This function returns columns that are either of object or categorical dtype,
    or numeric columns with fewer than `threshold` unique values.
    """
    cat_cols = []
    for col in df.columns:
        # If already object or categorical, add it.
        if df[col].dtype == 'object' or isinstance(df[col].dtype, pd.CategoricalDtype):
            cat_cols.append(col)
        # Otherwise, if numeric but with very few unique values, consider it categorical.
        elif np.issubdtype(df[col].dtype, np.number):
            if df[col].nunique() < threshold:
                cat_cols.append(col)
    return cat_cols


In [ ]:

# -------------------------------
# 2. Model training functions
# -------------------------------
def train_arima(y_train, forecast_steps):
    """
    Train a simple ARIMA model on the target series and forecast for the specified number of steps.
    Here we choose an order of (1,1,1) for demonstration.
    """
    model = ARIMA(y_train, order=(2,1,2))
    model_fit = model.fit()
    forecast = model_fit.forecast(steps=forecast_steps)
    return forecast  # Removed .values as forecast is already a numpy array

def train_sarima(y_train, forecast_steps):
    """
    SARIMA with order=(2,1,2) and seasonal_order=(1,1,1,24)
    for hourly data
    """
    model = SARIMAX(
        y_train, 
        order=(2,1,2), 
        seasonal_order=(1,1,1,24),         
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    model_fit = model.fit(disp=False, maxiter=500, method='powell')
    forecast = model_fit.forecast(steps=forecast_steps)
    return forecast


def train_prophet(train_df, test_df, date_column, target_column):
    """
    Train a Prophet model using only the target series.
    """
    df_train = train_df[[date_column, target_column]].rename(columns={date_column: 'ds', target_column: 'y'})
    model = Prophet(
        changepoint_prior_scale=0.1,    # Lower value for smoother trend
        seasonality_prior_scale=10.0,   # Slightly lower to prevent overfitting seasonality
        changepoint_range=0.9,
        n_changepoints=50,             # More potential changepoints for complex patterns
        yearly_seasonality=False,       # Set to True if data spans multiple years
        weekly_seasonality=True,
        daily_seasonality=True
    )
    model.add_seasonality(name='2H', period=1/12, fourier_order=12)
    model.add_seasonality(name='4H', period=1/6, fourier_order=12)
    model.add_seasonality(name='8H', period=1/3, fourier_order=12)
    model.fit(df_train)
    df_test = test_df[[date_column]].rename(columns={date_column: 'ds'})
    forecast = model.predict(df_test)
    return forecast['yhat'].values

def train_prophet_with_regressors(train_df, test_df, date_column, target_column, regressor_columns):
    """
    Train a Prophet model including extra regressors.
    The function first prepares the data by renaming the date and target columns.
    For any regressor that is categorical, we convert it to numeric codes.
    """
    # Prepare data for Prophet
    df_train = train_df[[date_column, target_column] + regressor_columns].rename(
        columns={date_column: 'ds', target_column: 'y'}
    )
    df_train = prepare_regressor_data_for_prophet(df_train, regressor_columns)
    model = Prophet(
        changepoint_prior_scale=0.1,    # Lower value for smoother trend
        seasonality_prior_scale=10.0,   # Slightly lower to prevent overfitting seasonality
        changepoint_range=0.9,
        n_changepoints=50,             # More potential changepoints for complex patterns
        yearly_seasonality=False,       # Set to True if data spans multiple years
        weekly_seasonality=True,
        daily_seasonality=True
    )
    model.add_seasonality(name='2H', period=1/12, fourier_order=12)
    model.add_seasonality(name='4H', period=1/6, fourier_order=12)
    model.add_seasonality(name='8H', period=1/3, fourier_order=12)
    for col in regressor_columns:
        model.add_regressor(col)
    model.fit(df_train)
    
    df_test = test_df[[date_column] + regressor_columns].rename(columns={date_column: 'ds'})
    df_test = prepare_regressor_data_for_prophet(df_test, regressor_columns)
    forecast = model.predict(df_test)
    return forecast['yhat'].values

def train_lgbm(X_train, y_train, X_test):

    model = LGBMRegressor(
        random_state=42,
        n_estimators=100,
        learning_rate=0.1,
        num_leaves=61,
        max_depth=8,
        boosting_type='gbdt'
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return preds

def train_xgboost(X_train, y_train, X_test):
    model = XGBRegressor(
        random_state=42,
        n_estimators=50,      # More boosting rounds
        learning_rate=0.1,    # Lower learning rate
        max_depth=7,           # Allow deeper trees for capturing complex interactions
        subsample=0.8,         # Use 80% of data for each tree to reduce overfitting
        colsample_bytree=0.8,
        verbosity=0
    )
    model.fit(X_train, y_train)
    return model.predict(X_test)

def train_catboost_df(train_df, y_train, test_df, cat_features):
    """
    Train CatBoost using DataFrame inputs and leverage its native handling of categorical features.
    The categorical features are cast to string to avoid type errors.
    """
    # Convert the categorical features to string
    train_df = train_df.copy()
    test_df = test_df.copy()
    for col in cat_features:
        train_df[col] = train_df[col].astype(str)
        test_df[col] = test_df[col].astype(str)
    
    model = CatBoostRegressor(
        iterations=50,
        learning_rate=0.1,
        depth=8,
        random_seed=42,
        verbose=0
    )
    model.fit(train_df, y_train, cat_features=cat_features)
    return model.predict(test_df)


# -------------------------------
# 3. Rolling window training pipeline
# -------------------------------
def rolling_window_forecasting(df, target_column, date_column, models_to_run,
                               train_window_days, test_window_days, rows_per_day,
                               optimization_metric='rmse', max_iterations=None,
                               max_clip=100, min_clip=0, ensemble_weights=None, starter=0):
    """
    Perform rolling-window forecasting.
    
    Parameters:
      - df: Original DataFrame (must include date and target columns).
      - target_column: Name of the target variable.
      - date_column: Name of the date/time column.
      - models_to_run: List of model names to run. (Allowed: 'arima', 'sarima', 'prophet', 'prophet_reg',
                        'lgbm', 'xgboost', 'catboost', and the baselines: 'baseline_last', 'baseline_mean7', 'baseline_yesterday',
                        and 'weighted_ensemble')
      - train_window_days: Number of days to use for training.
      - test_window_days: Number of days to use for testing.
      - rows_per_day: Number of rows per day (e.g. 96 for 15-minute intervals or 24 for hourly).
      - optimization_metric: (Not used inside the loop but can be used later for tuning.)
      - max_iterations: Maximum number of rolling window iterations to perform (default None means no limit).
      - max_clip: Maximum value to clip predictions.
      - min_clip: Minimum value to clip predictions.
      - ensemble_weights: Dictionary of weights for the ensemble forecast, e.g.
            {'prophet_reg': 0.25, 'lgbm': 0.25, 'catboost': 0.25, 'xgboost': 0.25}.
            If None, equal weights will be used.
      
    Returns:
      - predictions_df: DataFrame with test rows (with re-assigned, continuous test dates)
        plus a column per model with its predictions (all clipped between min_clip and max_clip)
        including the three baselines and the weighted ensemble.
    """
    # Ensure the date column is datetime and sort by date
    df = df.copy()
    cat_columns = get_categorical_columns(df.drop(columns=[target_column, date_column]), threshold=50)
    df[date_column] = pd.to_datetime(df[date_column])
    df.sort_values(by=date_column, inplace=True)
    
    predictions_list = []
    total_rows = df.shape[0]
    train_window = train_window_days * rows_per_day
    test_window = test_window_days * rows_per_day

    # Derive frequency string based on rows_per_day.
    # For example, if rows_per_day = 96, then interval = 1440/96 = 15 minutes.
    # For hourly data (rows_per_day = 24), freq_str will be "60min".
    freq_str = f"{1440 // rows_per_day}min"
    
    # Determine base test date from the first available test window.
    base_test_date = df.iloc[train_window][date_column]
    
    # Calculate total iterations available based on the data.
    total_iterations_available = ((total_rows - train_window - test_window) // test_window) + 1
    if max_iterations is not None:
        total_iterations = min(total_iterations_available, max_iterations)
    else:
        total_iterations = total_iterations_available
    
    from tqdm.notebook import tqdm
    pbar = tqdm(total=total_iterations, desc="Rolling Window Forecasting", unit="window")
    
    iteration_count = 0
    start_idx = starter
    print(start_idx)
    while start_idx + train_window + test_window <= total_rows and (max_iterations is None or iteration_count < max_iterations):
        # Get the current training and test windows
        train_df = df.iloc[start_idx: start_idx + train_window].copy()
        test_df = df.iloc[start_idx + train_window: start_idx + train_window + test_window].copy()
        
        # Reset the date column in the test window to be continuous
        # new_start = base_test_date + pd.Timedelta(days=iteration_count * test_window_days)
        # new_dates = pd.date_range(start=new_start, periods=len(test_df), freq=freq_str)
        # test_df[date_column] = new_dates
        
        # Prepare features for regression models
        X_train, y_train, X_test, y_test = prepare_features_for_regression(train_df, test_df, target_column, date_column)
        from sklearn.preprocessing import StandardScaler
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # For Prophet models, use the raw (or minimally transformed) data.
        regressor_columns = list(train_df.columns.drop([target_column, date_column]))
        train_df_prophet = train_df.copy()
        test_df_prophet = test_df.copy()
        
        # Start with the test window DataFrame (with reset dates)
        window_predictions = test_df.copy()
        
        # Loop over each model requested and add its predictions
        for model_name in models_to_run:
            if model_name == 'arima':
                preds = train_arima(train_df[target_column].values, len(test_df))
            elif model_name == 'sarima':
                preds = train_sarima(train_df[target_column].values, len(test_df))
            elif model_name == 'prophet':
                preds = train_prophet(train_df, test_df, date_column, target_column)
            elif model_name == 'prophet_reg':
                preds = train_prophet_with_regressors(train_df_prophet, test_df_prophet, date_column, target_column, regressor_columns)
            elif model_name == 'lgbm':
                preds = train_lgbm(X_train_scaled, y_train, X_test_scaled)
            elif model_name == 'xgboost':
                preds = train_xgboost(X_train_scaled, y_train, X_test_scaled)
            elif model_name == 'catboost':
                # Use the raw features (no one-hot encoding) for CatBoost.
                X_train_cat = train_df.drop(columns=[target_column, date_column])
                X_test_cat = test_df.drop(columns=[target_column, date_column])
                preds = train_catboost_df(X_train_cat, train_df[target_column].values, X_test_cat, cat_features=cat_columns)

            # Baseline 1: Last Value (naive forecast)
            elif model_name == 'baseline_last':
                baseline_last = np.empty(len(test_df))
                for i in range(len(test_df)):
                    if i == 0:
                        baseline_last[i] = train_df[target_column].iloc[-1]
                    else:
                        baseline_last[i] = test_df[target_column].iloc[i - 1]
                preds = baseline_last
            # Baseline 2: Mean of Last 7 Days for the Same Hour
            elif model_name == 'baseline_mean7':
                baseline_mean7 = np.empty(len(test_df))
                for i in range(len(test_df)):
                    current_timestamp = test_df[date_column].iloc[i]
                    current_hour = current_timestamp.hour
                    subset = train_df[(train_df[date_column] >= current_timestamp - pd.Timedelta(days=7)) &
                                      (train_df[date_column] < current_timestamp)]
                    subset_same_hour = subset[subset[date_column].dt.hour == current_hour]
                    if len(subset_same_hour) > 0:
                        baseline_mean7[i] = subset_same_hour[target_column].mean()
                    else:
                        subset_all = train_df[train_df[date_column].dt.hour == current_hour]
                        if len(subset_all) > 0:
                            baseline_mean7[i] = subset_all[target_column].mean()
                        else:
                            baseline_mean7[i] = train_df[target_column].mean()
                preds = baseline_mean7
            # Baseline 3: Yesterday's Value (seasonal naive)
            elif model_name == 'baseline_yesterday':
                baseline_yesterday = np.empty(len(test_df))
                for i in range(len(test_df)):
                    if i < rows_per_day:
                        baseline_yesterday[i] = train_df[target_column].iloc[-rows_per_day + i]
                    else:
                        baseline_yesterday[i] = test_df[target_column].iloc[i - rows_per_day]
                preds = baseline_yesterday
            # Weighted Ensemble of 'prophet_reg', 'lgbm', 'catboost', and 'xgboost'
            elif model_name == 'weighted_ensemble':
                try:
                    # If ensemble_weights not provided, use equal weights
                    if ensemble_weights is None:
                        weights = {'prophet_reg': 1, 'lgbm': 1, 'catboost': 1, 'xgboost': 1}
                    else:
                        weights = ensemble_weights
                    w1 = weights.get('prophet_reg', 1)
                    w2 = weights.get('lgbm', 1)
                    w3 = weights.get('catboost', 1)
                    w4 = weights.get('xgboost', 1)
                    sum_weights = w1 + w2 + w3 + w4
                    ensemble_preds = (w1 * window_predictions['prophet_reg'] +
                                      w2 * window_predictions['lgbm'] +
                                      w3 * window_predictions['catboost'] +
                                      w4 * window_predictions['xgboost']) / sum_weights
                    preds = ensemble_preds
                except KeyError:
                    print("Warning: Not all required models for weighted ensemble found. Skipping weighted ensemble.")
                    continue
            else:
                print(f"Warning: Model {model_name} not recognized. Skipping.")
                continue
            
            # Clip predictions between min_clip and max_clip
            window_predictions[model_name] = np.clip(preds, min_clip, max_clip)
        
        predictions_list.append(window_predictions)
        start_idx += test_window  # Slide the window forward
        iteration_count += 1
        pbar.update(1)
    
    pbar.close()
    predictions_df = pd.concat(predictions_list, axis=0)
    return predictions_df


In [ ]:
# -------------------------------
# 4. Evaluation and Visualization Pipeline
# -------------------------------
def evaluate_and_visualize(prediction_df, date_column, true_column, prediction_columns,
                           barplot_filename="barplot.html", lineplot_filename="lineplot.html"):
    """
    Given a DataFrame that includes:
      - a date column,
      - a true value column, and
      - one or more prediction columns (each named for the model),
    this function computes performance metrics and creates two Plotly figures:
      1) A single bar plot showing MAE, RMSE, MAPE, MAAPE (arctan MAPE), sMAPE, and MASE.
      2) A line plot showing true values and all predictions over time.
    
    The plots are saved as HTML files with the given filenames.
    
    Returns:
      - metrics_df: a DataFrame summarizing the metrics per model.
    """

    
    metrics_dict = {}
    for col in prediction_columns:
        y_true = prediction_df[true_column].values
        y_pred = prediction_df[col].values
        
        mae = mean_absolute_error(y_true, y_pred)
        rmse = np.sqrt(mean_squared_error(y_true, y_pred))
        
        # MAPE and MAAPE: Avoid division by zero by only computing on non-zero true values.
        nonzero_mask = y_true != 0
        if np.any(nonzero_mask):
            mape = np.mean(np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])) * 100
            maape = np.mean(np.arctan(np.abs((y_true[nonzero_mask] - y_pred[nonzero_mask]) / y_true[nonzero_mask])))
        else:
            mape = np.nan
            maape = np.nan

        # sMAPE: Avoid division by zero in the denominator.
        denominator = np.abs(y_true) + np.abs(y_pred)
        nonzero_denom_mask = denominator != 0
        if np.any(nonzero_denom_mask):
            smape = np.mean(2 * np.abs(y_pred[nonzero_denom_mask] - y_true[nonzero_denom_mask]) / denominator[nonzero_denom_mask]) * 100
        else:
            smape = np.nan

        # MASE: Use the naive forecast (lag 1) errors; if their mean is zero, set MASE to np.nan.
        naive_errors = np.abs(np.diff(y_true))
        mean_naive_error = np.mean(naive_errors) if np.mean(naive_errors) != 0 else np.nan
        mase = mae / mean_naive_error if mean_naive_error != 0 else np.nan
        
        metrics_dict[col] = {
            'MAE': mae,
            'RMSE': rmse,
            'MAPE': mape,
            'MAAPE': maape,
            'sMAPE': smape,
            'MASE': mase
        }
    
    metrics_df = pd.DataFrame(metrics_dict).T.reset_index().rename(columns={'index': 'Model'})
    
    # Create a single grouped bar plot with all metrics.
    fig_bar = go.Figure()
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['MAE'], name='MAE'))
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['RMSE'], name='RMSE'))
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['MAPE'], name='MAPE'))
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['MAAPE'], name='MAAPE'))
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['sMAPE'], name='sMAPE'))
    fig_bar.add_trace(go.Bar(x=metrics_df['Model'], y=metrics_df['MASE'], name='MASE'))
    
    fig_bar.update_layout(title="Performance Metrics per Model",
                          barmode='group',
                          xaxis_title="Model",
                          yaxis_title="Metric Value")
    fig_bar.write_html(barplot_filename)
    fig_bar.show()
    
    # Create a line plot for true vs. predicted values over time
    fig_line = go.Figure()
    fig_line.add_trace(go.Scatter(x=prediction_df[date_column], y=prediction_df[true_column],
                                  mode='lines', name='True', line=dict(width=2)))
    for col in prediction_columns:
        fig_line.add_trace(go.Scatter(x=prediction_df[date_column], y=prediction_df[col],
                                      mode='lines', name=col, line=dict(width=2)))
    fig_line.update_layout(title="True vs. Predicted Values", xaxis_title=date_column, yaxis_title=true_column)
    fig_line.write_html(lineplot_filename)
    fig_line.show()
    
    return metrics_df

# -------------------------------
# 5. Main forecasting pipeline function
# -------------------------------
def run_forecasting_pipeline(df, target_column, date_column, comparison_columns, models_to_run,
                             optimization_metric='rmse', train_window_days=90,
                             test_window_days=7, rows_per_day=24,max_iterations=3, max_clip=100, start_idx=0,
                             barplot_filename="barplot.html", lineplot_filename="lineplot.html"):
    """
    End-to-end pipeline:
      - Prepares the data,
      - Runs rolling-window training for the requested models,
      - Merges predictions with the original features (for test windows),
      - Evaluates and visualizes performance.
    
    Returns:
      - predictions_df: DataFrame with test rows and one column per model prediction.
      - metrics_df: Summary of evaluation metrics per model.
    """
    predictions_df = rolling_window_forecasting(
        df, target_column, date_column, models_to_run,
        train_window_days, test_window_days, rows_per_day,
        optimization_metric, max_iterations, max_clip, starter=start_idx
    )
    candidates = models_to_run + comparison_columns
    # Evaluate using the provided true target values and model prediction columns.
    metrics_df = evaluate_and_visualize(
        predictions_df, date_column, target_column, candidates,
        barplot_filename, lineplot_filename
    )
    
    return predictions_df, metrics_df


# -------------------------------
# 6. aggregate forecasting pipeline function
# -------------------------------
def run_aggregate_pipeline(df, target_column, date_column, comparison_columns, models_to_run,
                             optimization_metric='rmse', train_window_days=90,
                             test_window_days=7, rows_per_day=24,max_iterations=3, max_clip=100, start_idx=0,
                             barplot_filename="barplot.html", lineplot_filename="lineplot.html"):
    """
    End-to-end pipeline:
      - Prepares the data,
      - Runs rolling-window training for the requested models,
      - Merges predictions with the original features (for test windows),
      - Evaluates and visualizes performance.
    
    Returns:
      - predictions_df: DataFrame with test rows and one column per model prediction.
      - metrics_df: Summary of evaluation metrics per model.
    """
    predictions = []
    for plant in df.plant_name.unique():
        df_ex = df.copy()
        df_ex = df_ex[df_ex.plant_name == plant]
    
        predictions_df = rolling_window_forecasting(
            df_ex, target_column, date_column, models_to_run,
            train_window_days, test_window_days, rows_per_day,
            optimization_metric, max_iterations, max_clip, starter=start_idx
        )
        predictions.append(predictions_df)
    
    all_cols = models_to_run + comparison_columns + [date_column] + [target_column]
    df_pred_all = pd.concat(predictions)[all_cols]
    df_pred_agg = df_pred_all.groupby(date_column, as_index=False).sum()
    
    
    candidates = models_to_run + comparison_columns
    # Evaluate using the provided true target values and model prediction columns.
    metrics_df = evaluate_and_visualize(
        df_pred_agg, date_column, target_column, candidates,
        barplot_filename, lineplot_filename
    )
    
    return predictions_df, metrics_df




In [ ]:
df_case = df_c4.copy()
df_case = df_case[df_case.plant_name == 1].drop(columns=["plant_name", "capacity"])

# Assume target is "production" and date is "effectivedate"

# Define which models to run – choose any subset of:
# 'arima', 'prophet', 'prophet_reg', 'lgbm', 'xgboost', 'catboost'
models_to_run = ['baseline_yesterday', 'baseline_mean7', 'baseline_last','arima', "sarima", 'prophet', 'prophet_reg', 'lgbm', 'xgboost', 'catboost','weighted_ensemble']
models_to_run = ["lgbm"]
# Run the forecasting pipeline.
predictions, metrics = run_forecasting_pipeline(
    df=df_case,
    target_column="production",
    date_column="effectivedate",
    comparison_columns = ["provider1_fc1200","provider2_fc1200"],
    models_to_run=models_to_run,
    optimization_metric='mae',   # or 'mae'
    train_window_days=30,
    test_window_days=7,
    rows_per_day=24,               
    max_iterations = 2,
    barplot_filename="forecast_barplota.html",
    lineplot_filename="forecast_lineplot.html",
    max_clip=23.3,
    start_idx=20000
)

# Optionally, save the predictions DataFrame
predictions.to_csv("forecast_predictions.csv", index=False)
print("Forecasting pipeline completed. Metrics:")
print(metrics)

In [ ]:
predictions

# Case Runs

Case 1: Provider 1 & Provider 2 + Supplement Forecast Data = Last FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 2: Provider 1 & Provider 2 + Supplement Forecast Data + Meteo Data = Last FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 3v1: Provider 1 & Provider 2 + Meteo Data = Dayahead FC
- Available Plants: A, B, C, D, E, F, G, H, J, K

Case 3v2: Provider 1 & Provider 2 = Dayahead FC
- Available Plants: A, B, C, D, E, F, G, H, J, K


In [ ]:
cap_list = list(df.capacity.unique())
cap_list.sort()
cap_list

In [ ]:
len(df[df.plant_name==3])

For Plants: 6 (112.6), 2 (74.8), 5 (57.0), 3 (50.5) 

- Case 1,2,3,4 MAE
- Case 1,2,3,4 RMSE
- Training 180 Days
- Test 15 Days
- 2 Iterations with start_idx = 0
- 2 Iterastions with start_idx = 20000

In [ ]:
cases

In [ ]:
opt = 'mae'
case_name = 'case_2'
df_case = cases[case_name].copy()
# df_case = df_case[(df_case.plant_name == 6) | (df_case.plant_name == 2)]
if case_name == "case_3" or  case_name == "case_4":
    comp_cols = ["provider1_fc1200","provider2_fc1200"]
else:
    comp_cols = ["provider1_fc0","provider2_fc0"]
starter = 20000
models_to_run = ['baseline_yesterday', 'baseline_mean7', 'baseline_last','arima', "sarima", 'prophet', 
                                 'prophet_reg', 'lgbm', 'xgboost', 'catboost','weighted_ensemble']

preds = run_aggregate_pipeline(df=df_case,
                    target_column="production",
                    date_column="effectivedate",
                    comparison_columns = comp_cols,
                    models_to_run=models_to_run,
                    optimization_metric=opt,   # or 'mae'
                    train_window_days=180,
                    test_window_days=15,
                    rows_per_day=24,               
                    max_iterations = 2,
                    barplot_filename=f"forecast_barplot_{opt}_{case_name}_idx{starter}.html",
                    lineplot_filename=f"forecast_lineplot_{opt}_{case_name}_idx{starter}.html",
                    max_clip=150,
                    start_idx=starter)

In [ ]:
for case_name,case_df in cases.items():
    if case_name == "case_3" or  case_name == "case_4":
        comp_cols = ["provider1_fc1200","provider2_fc1200"]
    else:
        comp_cols = ["provider1_fc0","provider2_fc0"]
    for plant in [6,2,5,3]:
        for opt in ["mae","rmse"]:
            for starter in [0,20000]:
                # Define which models to run – choose any subset of:
                # 'arima', 'prophet', 'prophet_reg', 'lgbm', 'xgboost', 'catboost'
                models_to_run = ['baseline_yesterday', 'baseline_mean7', 'baseline_last','arima', "sarima", 'prophet', 
                                 'prophet_reg', 'lgbm', 'xgboost', 'catboost','weighted_ensemble']
#                 models_to_run = ["lgbm"]
                df_ex = case_df.copy()
                df_ex = df_ex[df_ex.plant_name == plant]
                # Run the forecasting pipeline.
                predictions, metrics = run_forecasting_pipeline(
                    df=df_ex,
                    target_column="production",
                    date_column="effectivedate",
                    comparison_columns = comp_cols,
                    models_to_run=models_to_run,
                    optimization_metric=opt,   # or 'mae'
                    train_window_days=180,
                    test_window_days=15,
                    rows_per_day=24,               
                    max_iterations = 2,
                    barplot_filename=f"forecast_barplot_{opt}_{case_name}_plant{plant}_idx{starter}.html",
                    lineplot_filename=f"forecast_lineplot_{opt}_{case_name}_plant{plant}_idx{starter}.html",
                    max_clip=150,
                    start_idx=starter
                )
                # Optionally, save the predictions DataFrame
                predictions.to_csv(f"forecast_predictions_{opt}_{case_name}_plant{plant}_idx{starter}.csv", index=False)
                print("Forecasting pipeline completed. Metrics:")
                print(metrics)


In [ ]:
df_case = df_c4.copy()
df_case = df_case[df_case.plant_name == 1].drop(columns=["plant_name", "capacity"])

# Assume target is "production" and date is "effectivedate"

# Define which models to run – choose any subset of:
# 'arima', 'prophet', 'prophet_reg', 'lgbm', 'xgboost', 'catboost'
models_to_run = ['baseline_yesterday', 'baseline_mean7', 'baseline_last','arima', "sarima", 'prophet', 'prophet_reg', 'lgbm', 'xgboost', 'catboost','weighted_ensemble']
# models_to_run = ["sarima"]
# Run the forecasting pipeline.
predictions, metrics = run_forecasting_pipeline(
    df=df_case,
    target_column="production",
    date_column="effectivedate",
    comparison_columns = ["provider1_fc1200","provider2_fc1200"],
    models_to_run=models_to_run,
    optimization_metric='mae',   # or 'mae'
    train_window_days=30,
    test_window_days=7,
    rows_per_day=24,               
    max_iterations = 1,
    barplot_filename="forecast_barplota.html",
    lineplot_filename="forecast_lineplot.html",
    max_clip=23.3
)

# Optionally, save the predictions DataFrame
predictions.to_csv("forecast_predictions.csv", index=False)
print("Forecasting pipeline completed. Metrics:")
print(metrics)